# Bussiness Case - To check if the loan will default or not

## Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option("display.max_columns",None)
pd.set_option("display.max_columns",None)
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from imblearn.over_sampling import SMOTE

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Loading the Data

In [3]:
df1 = pd.read_csv('/content/drive/MyDrive/loan Defaulter/application_train.csv')

In [4]:
df2 = pd.read_csv('/content/drive/MyDrive/loan Defaulter/bureau.csv')

In [5]:
df3 = pd.read_csv('/content/drive/MyDrive/loan Defaulter/bureau_balance.csv')

In [6]:
df4 = pd.read_csv('/content/drive/MyDrive/loan Defaulter/credit_card_balance.csv')

In [7]:
df5 = pd.read_csv('/content/drive/MyDrive/loan Defaulter/installments_payments.csv')

In [8]:
df6 = pd.read_csv('/content/drive/MyDrive/loan Defaulter/POS_CASH_balance.csv')

In [9]:
df7 = pd.read_csv('/content/drive/MyDrive/loan Defaulter/previous_application.csv')

In [10]:
df1_1 = df1.groupby(df1['SK_ID_CURR'], as_index = False).mean()

In [11]:
df2_2 = df2.groupby(df2['SK_ID_BUREAU'], as_index = False).mean()

In [12]:
df3_3 = df3.groupby(df3['SK_ID_BUREAU'], as_index = False).mean()

In [13]:
df4_4 = df4.groupby(df4['SK_ID_CURR'], as_index = False).mean()

In [14]:
df5_5 = df5.groupby(df5['SK_ID_CURR'], as_index = False).mean()

In [15]:
df6_6 = df6.groupby(df6['SK_ID_CURR'], as_index = False).mean()

In [16]:
df7_7 = df7.groupby(df7['SK_ID_CURR'], as_index = False).mean()

In [17]:
df8_8 = pd.merge(df2_2, df3_3, on='SK_ID_BUREAU', how='left')

In [18]:
df4_4.drop(['SK_ID_PREV'], axis=1, inplace=True)
df5_5.drop(['SK_ID_PREV'], axis=1, inplace=True)
df6_6.drop(['SK_ID_PREV', 'MONTHS_BALANCE'], axis=1, inplace=True)
df7_7.drop(['SK_ID_PREV'], axis=1, inplace=True)

In [19]:
df9_9 = pd.merge(df8_8,df4_4, on='SK_ID_CURR', how='left')

In [20]:
df10_10 = pd.merge(df9_9,df5_5, on='SK_ID_CURR',how='left')

In [21]:
df11_11 = pd.merge(df10_10,df6_6, on='SK_ID_CURR',how='left')

In [22]:
df12_13 = pd.merge(df11_11,df6_6, on='SK_ID_CURR',how='left')

In [23]:
df13_13 = pd.merge(df12_13, df7_7, on='SK_ID_CURR',how='left')

## Final Data Set

In [24]:
df = pd.merge(df13_13, df1_1, on='SK_ID_CURR',how='left')

# Feature Engineering

## Missing Value Treatment

In [25]:
## Finding Missing Value

mis_val = df.isnull().sum()
mis_val_percent = 100 * df.isnull().sum() / len(df)
mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
mis_val_table_ren_columns = mis_val_table.rename(columns = {0 : 'mis_val_count', 1 : 'mis_val_percent'})
mis_val_table_ren_columns = mis_val_table_ren_columns[mis_val_table_ren_columns.iloc[:,1] != 0].sort_values('mis_val_percent', ascending=False).round(1)
print ("Total Number of Features: " + str(df.shape[1]) + "\n" + "Total Number of Features with Null Value: " + str(mis_val_table_ren_columns.shape[0]))
print(mis_val_table_ren_columns.head(50))

Total Number of Features: 173
Total Number of Features with Null Value: 166
                            mis_val_count  mis_val_percent
RATE_INTEREST_PRIMARY             1685511             98.2
RATE_INTEREST_PRIVILEGED          1685511             98.2
CNT_DRAWINGS_OTHER_CURRENT        1325940             77.2
AMT_DRAWINGS_POS_CURRENT          1325940             77.2
AMT_DRAWINGS_OTHER_CURRENT        1325940             77.2
CNT_DRAWINGS_POS_CURRENT          1325940             77.2
AMT_DRAWINGS_ATM_CURRENT          1325940             77.2
CNT_DRAWINGS_ATM_CURRENT          1325940             77.2
AMT_PAYMENT_CURRENT               1325835             77.2
COMMONAREA_MEDI                   1248808             72.8
COMMONAREA_MODE                   1248808             72.8
COMMONAREA_AVG                    1248808             72.8
NONLIVINGAPARTMENTS_MEDI          1242152             72.4
NONLIVINGAPARTMENTS_MODE          1242152             72.4
NONLIVINGAPARTMENTS_AVG           12421

In [26]:
## Dropping the columns with more than 70% Null Values


df.drop(['RATE_INTEREST_PRIMARY', 'RATE_INTEREST_PRIVILEGED','AMT_CREDIT_y', 'CNT_INSTALMENT_x', 'CNT_INSTALMENT_FUTURE_x', 'AMT_GOODS_PRICE_y', 
         'HOUR_APPR_PROCESS_START_y', 'SK_DPD_x', 'SK_DPD_DEF_x','AMT_ANNUITY_x',	'MONTHS_BALANCE_x', 'SK_ID_BUREAU', 'CNT_DRAWINGS_OTHER_CURRENT', 'AMT_DRAWINGS_POS_CURRENT', 'AMT_DRAWINGS_OTHER_CURRENT',
         'CNT_DRAWINGS_POS_CURRENT', 'AMT_DRAWINGS_ATM_CURRENT', 'CNT_DRAWINGS_ATM_CURRENT', 'AMT_PAYMENT_CURRENT', 'COMMONAREA_MEDI', 
         'COMMONAREA_MODE', 'COMMONAREA_AVG', 'NONLIVINGAPARTMENTS_MEDI', 'NONLIVINGAPARTMENTS_MODE', 'NONLIVINGAPARTMENTS_AVG', 
         'AMT_ANNUITY_x', 'LIVINGAPARTMENTS_AVG', 'LIVINGAPARTMENTS_MEDI', 'LIVINGAPARTMENTS_MODE', 'FLOORSMIN_MODE', 'FLOORSMIN_AVG', 'FLOORSMIN_MEDI'
         ], axis=1, inplace=True)

In [27]:
## Imputing the missing values 

for columns in df:
      if df[columns].dtypes == object:
        imputer = SimpleImputer(missing_values = np.nan,strategy = 'most_frequent')

      else:
        imputer = SimpleImputer(missing_values = np.nan,strategy = 'median')

      imputer.fit(df[[columns]])
    
      df[columns] = imputer.transform(df[[columns]])

## Handling the Categorical Data

In [28]:
## Handling the Categoical Data

lc = LabelEncoder()
for columns in df:
  df[columns] = lc.fit_transform(df[columns])

## Creating X & Y

In [29]:
## Creating X & y

y = df.TARGET
X = df.drop(['TARGET'], axis=1)

## Train & Test Split

In [30]:
## Training and Testing Data

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)

## Balancing the Data

In [31]:
## Balancing the Data

smote = SMOTE()

X_smote, y_smote = smote.fit_resample(X_train, y_train)

## Feature Selection

In [32]:
df.corr()

,SK_ID_CURR,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,DAYS_CREDIT_UPDATE,MONTHS_BALANCE_y,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_CURRENT,AMT_INST_MIN_REGULARITY,AMT_PAYMENT_TOTAL_CURRENT,AMT_RECEIVABLE_PRINCIPAL,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT,SK_DPD_y,SK_DPD_DEF_y,CNT_INSTALMENT_y,CNT_INSTALMENT_FUTURE_y,SK_DPD,SK_DPD_DEF,AMT_ANNUITY_y,AMT_APPLICATION,AMT_CREDIT_x,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE_x,HOUR_APPR_PROCESS_START_x,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,DAYS_DECISION,SELLERPLACE_AREA,CNT_PAYMENT,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_ANNUITY,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,LANDAREA_AVG,LIVINGAREA_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,LANDAREA_MODE,LIVINGAREA_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,LANDAREA_MEDI,LIVINGAREA_MEDI,NONLIVINGAREA_MEDI,TOTALAREA_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
SK_ID_CURR,1.000000,0.000266,0.000369,0.000466,-0.000580,-0.001046,-0.000388,0.000897,-0.000112,-0.000497,0.000699,0.000372,0.000289,-0.000730,-0.001031,0.000819,-0.000712,0.001727,-0.000745,-0.000749,-0.000750,0.001398,-0.000180,-0.001347,0.000687,0.000877,0.000830,0.002388,0.002237,0.000453,-0.002390,-0.000859,-0.000966,0.000453,-0.002390,0.002857,0.001122,0.000528,0.004447,0.001501,0.006078,0.001613,0.002002,-0.000889,-0.000949,-0.000154,0.001053,0.000823,-0.001273,-0.000711,-0.000961,0.003721,-0.002483,-0.000627,0.003070,-0.000779,0.000573,-0.001909,0.002296,0.000552,0.001486,-0.000356,NaN,-0.000151,0.001737,0.000210,0.006237,-0.000737,-0.001860,-0.001807,-0.001959,-0.000901,0.003034,0.004539,-0.000981,-0.002634,-0.001952,-0.000238,0.001267,0.000374,0.001727,-0.003074,0.004518,0.005166,0.004107,-0.001798,0.005214,-0.000945,0.002540,0.000417,0.001618,-0.002817,0.004313,0.004744,0.004463,-0.001491,0.004893,-0.000846,0.002654,0.000224,0.001917,-0.002909,0.004335,0.005138,0.004463,-0.001226,0.004935,-0.001143,0.002839,0.000287,0.002566,-0.000131,0.000897,-0.000162,0.001287,0.000198,NaN,-0.005460,-0.003518,0.000906,0.000147,-0.002233,0.003567,-0.001401,0.001112,-0.001670,-0.001831,0.003380,0.001629,0.003555,-0.001560,0.002625,0.000310,0.004621,0.000468,0.000636,-0.001604,-0.001246,0.002453,0.003028,-0.003747,0.000366
DAYS_CREDIT,0.000266,1.000000,-0.025019,0.388022,0.711424,-0.084498,-0.030460,0.130379,0.307

## Model Creation

In [33]:
from sklearn.ensemble import RandomForestClassifier

In [34]:
rf = RandomForestClassifier(n_estimators=10)
rf.fit(X_smote, y_smote)

RandomForestClassifier(n_estimators=10)

In [35]:
y_pred = rf.predict(X_test)

In [36]:
from sklearn.metrics import classification_report,f1_score
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.99      1.00      1.00    320337
           1       0.98      0.93      0.95     22949

    accuracy                           0.99    343286
   macro avg       0.99      0.96      0.97    343286
weighted avg       0.99      0.99      0.99    343286



In [37]:
f_Score=f1_score(y_test,y_pred)
f_Score

0.9530597198328828